### Поиск связей в ЗУ и ОКС v5 в базе DK_V_BUILDING и DK_V_PARCEL (Multiprocessing)

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import re
import cx_Oracle #before import, execute from Anaconda prompt: conda install cx_Oracle
import multiprocessing

from IPython.core.display import display, HTML, display_html
display(HTML("<style>.container { width:99% !important; }</style>")) # set full window wide cells (only in this notebook)

In [10]:
from notebook.auth import passwd
passwd()

Enter password: ········
Verify password: ········


'sha1:4d9f0d14ac61:afb24ed03db3484db429be094b18d2d790d8d81c'

In [2]:
# Для векторизации импортируем CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Импортируем библиотеки для иерархической кластеризации
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 120)

In [5]:
#pd.reset_option('display.max_columns')
#pd.reset_option('display.max_rows')
#pd.reset_option('display.max_colwidth')

In [6]:
#Функция для вывода срезов датафрейма в нескольких колонках одной ячейки
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

#### Чтение из Oracle базы в Pandаs датафрейм

In [ ]:
# Подключаемся к базе Oracle
connection = cx_Oracle.connect('READ_USER/READ_USER@172.16.108.27:1521/jobdbp_1')
cursor = connection.cursor()

In [12]:
%%time
#PARCELS
df_l = pd.read_sql(sql='''select DISTINCT 
l.CADNUM,l.CADCOST,l.AREA,
case when l.ENTSPAT_ID is null then 0 else 1 end as ORDINATE,
l.PARCELINEGRP as INEGRP,l.DTCR,l.OKATO,l.ADDRESS,l.AO,l.AO2 
from DK_V_PARCEL_20180801 l 
where l.CADNUM is not null and l.ACTUAL=1''', con=connection)

Wall time: 58.3 s


In [17]:
df_l['L0B1'] = 0
print(df_l.shape)
display_side_by_side(df_l.head(), df_l.tail())

(351614, 11)


,CADNUM,CADCOST,AREA,ORDINATE,INEGRP,DTCR,OKATO,ADDRESS,AO,AO2,L0B1
0,50:00:0000000:27,513901.00,50.0,1,0,2012-07-06,45297581000,"г Щербинка, (город Москва, г. Щербинка, ул. Ко...",ТиНАО,Щербинка,0
1,50:10:0010101:72,37838095.62,3783.0,1,0,2011-10-29,45298578000,"р-н Химкинский, г Химки, (Московская область, ...",ТиНАО,Троицк,0
2,50:10:0010118:1,453039.20,10.0,0,0,2000-09-26,45293558000,"ул Молодежная, (обл. Московская, р-н Химкински...",Москва,ЮЗАО,0
3,50:10:0010301:2,497197.20,10.0,1,0,2003-11-25,45286585000,"р-н Химкинский, тер автомагистраль Москва- С.П...",Москва,ЦАО,0
4,50:10:0040202:28,22730322.06,1013.0,1,1,2009-06-19,45277584000,"ул Комсомольская, вл 4, строен 1, (г. Москва, ...",Москва,САО,0
,CADNUM,CADCOST,AREA,ORDINATE,INEGRP,DTCR,OKATO,ADDRESS,AO,AO2,L0B1
351609,77:22:0040408:52,594105.66,489.0,1,1,2017-09-04,45298553105,"п Вороновское, с Богоявление, тер СНТ Световод...",ТиНАО,Вороновское,0
351610,77:22:0040408:63,602610.24,496.0,1,1,2017-11-20,45298553000,"п Вороновское, тер СНТ Световод, (Москва, п Во...",ТиНАО,Вороновское,0
351611,77:22:0040411:264,691378.08,516.0,1,1,2016-08-09,45298558000,"п Кленовское, (Москва, п Кленовское, вблизи д....",ТиНАО,Кленовское,0
351612,77:22:0040411:303,686018.56,512.0,1,1,2017-12-22,45298558000,"п Кленовское, (Москва, п Кленовское, вблизи д....",ТиНАО,Кленовское,0


In [31]:
#df_l[df_l['ADDRESS'].isna()==True].sort_values('AREA')
df_l.drop(df_l[df_l['ADDRESS'].isna()==True].index, inplace=True)

In [32]:
%%time
#BUILDINGS
df_b = pd.read_sql(sql='''select DISTINCT 
b.CADNUM,b.UNOM,b.CADCOST,b.AREA,b.ORDINATE,b.BUILDINEGRP as INEGRP,b.REG_NUMR_DTCR as DTCR,b.EXPCHAR_YEARBUILD,b.EXPCHAR_YEARUSED,b.NAME,b.LIVING,b.ADDR_ID,b.OKATO,b.ADDRESS,b.AO,b.AO2,
a.STREETNM 
from DK_V_BUILDING_20180801 b 
left join dk_v_building_to_land_20180801 l 
on b.CADNUM=l.BUILD_CADNUM and l.SOURCE = 'GKN' 
left join ASUR_RR2_20180801.T$RR#ADDR a 
on b.ADDR_ID = a.ADDR_ID 
where b.REALTYTYPE='Здание' and b.ACTUAL=1 and l.BUILD_CADNUM is null and nvl(b.AREA,0)>0 and b.LIVING=0''', con=connection)

Wall time: 4min 4s


In [35]:
df_b.shape

(102908, 17)

In [58]:
# Сколько null значений в каждой колонке датафрейма
def df_na(df_x):
    for i in df_x.columns:
        print(len(df_x[df_x[i].isna()]), i, end='   ')
    print('\n')

df_na(df_l)
df_na(df_b)

0 CADNUM   1893 CADCOST   12 AREA   0 ORDINATE   0 INEGRP   24 DTCR   6343 OKATO   0 ADDRESS   0 AO   0 AO2   0 L0B1   

0 CADNUM   93337 UNOM   1 CADCOST   0 AREA   0 ORDINATE   0 INEGRP   0 DTCR   54444 EXPCHAR_YEARBUILD   85890 EXPCHAR_YEARUSED   10347 NAME   0 LIVING   0 ADDR_ID   0 OKATO   0 ADDRESS   0 AO   0 AO2   51046 STREETNM   0 L0B1   



In [59]:
df_b['L0B1'] = 1 # Set building flag
df_b[500:501]

,CADNUM,UNOM,CADCOST,AREA,ORDINATE,INEGRP,DTCR,EXPCHAR_YEARBUILD,EXPCHAR_YEARUSED,NAME,LIVING,ADDR_ID,OKATO,ADDRESS,AO,AO2,STREETNM,L0B1
500,77:13:0000000:1057,None,4513361.47,129.8,0,1,2014-08-28,None,None,Пожарное Депо,0,251006686,45297581001,"г Щербинка, ул Дорожная, д 5, (г.Москва, г.Щер...",ТиНАО,Щербинка,Дорожная,1


df_b.drop('L0-B1' ,axis=1, inplace=True)
df_b.head(1)

In [ ]:
# Объединим здания и участки в одну табл.
df = pd.concat([ df_b[['CADNUM','CADCOST','AREA','ORDINATE','INEGRP','DTCR','OKATO','ADDRESS','AO','AO2','STREETNM','L0B1']], df_l ], sort=False)
display_side_by_side(df[['L0B1','CADNUM','ADDRESS','STREETNM']].head(), df[['L0B1','CADNUM','ADDRESS','STREETNM']].tail())

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453921 entries, 0 to 351613
Data columns (total 12 columns):
CADNUM      453921 non-null object
CADCOST     452027 non-null float64
AREA        453909 non-null float64
ORDINATE    453921 non-null int64
INEGRP      453921 non-null int64
DTCR        453897 non-null datetime64[ns]
OKATO       447578 non-null object
ADDRESS     453921 non-null object
AO          453921 non-null object
AO2         453921 non-null object
STREETNM    51862 non-null object
L0B1        453921 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 45.0+ MB


In [63]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453921 entries, 0 to 453920
Data columns (total 12 columns):
CADNUM      453921 non-null object
CADCOST     452027 non-null float64
AREA        453909 non-null float64
ORDINATE    453921 non-null int64
INEGRP      453921 non-null int64
DTCR        453897 non-null datetime64[ns]
OKATO       447578 non-null object
ADDRESS     453921 non-null object
AO          453921 non-null object
AO2         453921 non-null object
STREETNM    51862 non-null object
L0B1        453921 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 41.6+ MB


In [64]:
#WRITE!!!
#df.to_csv(u'DK_V_PARCEL_TO_BUILDINGS.csv', sep=';', encoding = 'cp1251', index=False)
df.to_csv(u'DK_V_PARCEL_TO_BUILDINGS+STREETNM.csv', sep=';', encoding = 'cp1251', index=False)

In [7]:
#READ!
df = pd.read_csv(u'DK_V_PARCEL_TO_BUILDINGS.csv', sep=';', encoding = 'cp1251', low_memory=False)
#df = pd.read_csv(u'DK_V_PARCEL_TO_BUILDINGS+STREETNM.csv', sep=';', encoding = 'cp1251', low_memory=False

%%time
#Заполним NaN значения нулями и приведем тип данных в заданных колонках к int
print('Найдено NaN, обнулено и приведено к типу int:')
for col in ['UNOM','EXPCHAR_YEARBUILD','EXPCHAR_YEARUSED']:
    print( sum(df_du[col].isna()), col )
    df_du[col].fillna(0, inplace=True)
    df_du[col] = df_du[col].astype(int)

In [8]:
df = df[df['AO']=='Москва']
print(df.shape)
df.head()

(145218, 12)


,CADNUM,CADCOST,AREA,ORDINATE,INEGRP,DTCR,OKATO,ADDRESS,AO,AO2,STREETNM,L0B1
19,77:02:0023007:1016,1987568.12,36.6,1,0,2012-05-24,4.528055e+10,"р-н Алексеевский, б-р Ракетный, д 16, строен 1",Москва,СВАО,Ракетный,1
20,77:02:0022005:1016,53836806.24,316.0,1,0,2012-05-24,4.528057e+10,"р-н Останкинский, ул Аргуновская, д 3а, строен 1",Москва,СВАО,Аргуновская,1
21,77:02:0020003:1059,11470063.20,220.8,1,0,2012-05-24,4.528056e+10,"р-н Бутырский, ул Фонвизина, д 7, строен 1",Москва,СВАО,Фонвизина,1
22,77:09:0002001:1034,40699724.48,353.4,1,0,2012-05-27,4.527757e+10,"р-н Дмитровский, ул Вагоноремонтная, д 6, строен 1",Москва,САО,Вагоноремонтная,1
35,77:03:0003001:2050,29951747.70,376.4,0,1,2017-02-16,4.526359e+10,"ул Сокольнический Вал, д 37А, строение 11, (г.Москва, ул.Сокольнический Вал, д.37А, строен.11)",Москва,ВАО,Сокольнический Вал,1


In [9]:
df['STREETNM'].fillna('', inplace=True)
len(df[df['STREETNM'].str.contains(' М\.| Б\.')])

728

In [10]:
# Set building flag
len(df[df['L0B1']==1])

27580

In [11]:
# How much punctuation in addresses?
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
for i in punctuation:
    print(i, len(df[df['ADDRESS'].str.contains(i, regex=False)]), end='\t\t')

! 0		" 4847		# 0		$ 0		% 0		& 0		' 2		( 130606		) 130606		* 0		+ 182		, 143554		- 45972		. 71615		/ 20050		: 9285		; 882		< 7		= 0		> 7		? 0		@ 0		[ 1		\ 3		] 1		^ 0		_ 179		` 0		{ 0		| 0		} 0		~ 1		

In [12]:
# Clean address from punctuation
df['STR'] = df['ADDRESS'].str.lower().replace('\(|\)|\,|\.|\<|\>|\[|\]|\_|\:|\;|\"', ' ', regex=True)

In [13]:
# How much punctuation in addresses cleaned?
punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
for i in punctuation:
    print(i, len(df[df['STR'].str.contains(i, regex=False)]), end='\t\t')

! 0		" 0		# 0		$ 0		% 0		& 0		' 2		( 0		) 0		* 0		+ 182		, 0		- 45972		. 0		/ 20050		: 0		; 0		< 0		= 0		> 0		? 0		@ 0		[ 0		\ 3		] 0		^ 0		_ 0		` 0		{ 0		| 0		} 0		~ 1		

In [14]:
df.head()

,CADNUM,CADCOST,AREA,ORDINATE,INEGRP,DTCR,OKATO,ADDRESS,AO,AO2,STREETNM,L0B1,STR
19,77:02:0023007:1016,1987568.12,36.6,1,0,2012-05-24,4.528055e+10,"р-н Алексеевский, б-р Ракетный, д 16, строен 1",Москва,СВАО,Ракетный,1,р-н алексеевский б-р ракетный д 16 строен 1
20,77:02:0022005:1016,53836806.24,316.0,1,0,2012-05-24,4.528057e+10,"р-н Останкинский, ул Аргуновская, д 3а, строен 1",Москва,СВАО,Аргуновская,1,р-н останкинский ул аргуновская д 3а строен 1
21,77:02:0020003:1059,11470063.20,220.8,1,0,2012-05-24,4.528056e+10,"р-н Бутырский, ул Фонвизина, д 7, строен 1",Москва,СВАО,Фонвизина,1,р-н бутырский ул фонвизина д 7 строен 1
22,77:09:0002001:1034,40699724.48,353.4,1,0,2012-05-27,4.527757e+10,"р-н Дмитровский, ул Вагоноремонтная, д 6, строен 1",Москва,САО,Вагоноремонтная,1,р-н дмитровский ул вагоноремонтная д 6 строен 1
35,77:03:0003001:2050,29951747.70,376.4,0,1,2017-02-16,4.526359e+10,"ул Сокольнический Вал, д 37А, строение 11, (г.Москва, ул.Сокольнический Вал, д.37А, строен.11)",Москва,ВАО,Сокольнический Вал,1,ул сокольнический вал д 37а строение 11 г москва ул сокольнический вал д 37а строен 11


In [15]:
#WRITE!!!
#df.to_csv(u'DK_V_PARCEL_TO_BUILDINGS_MOS+STR.csv', sep='\t', encoding = 'cp1251', index=False)

In [16]:
# Список улиц Москвы с ресурса Публичная кадастровая карта России https://egrp365.ru/reestr?id=g1hIHC
df_streets = pd.read_excel('egrp365_streets.xlsx')
print(df_streets.shape)
display_side_by_side(df_streets.head(), df_streets.tail(), df_streets[df_streets['EGRP365_STREETS'].str.len()<=10])

(3637, 1)


,EGRP365_STREETS
0,аллея 1 Маевки
1,проезд 1-й Ходынский
2,улица 10-летия Октября
3,улица 1812 года / 1812 Года
4,улица 1905 года / 1905 Года
,EGRP365_STREETS
3632,аллея Яузская
3633,площадь Яузские Ворота
3634,бульвар Яузский
3635,улица Яхромская


In [17]:
# Чистка улиц от скобок и дробей
df_streets['EGRP365_STREETS_CLEAN'] = df_streets['EGRP365_STREETS'].str.lower().replace('\(|\)|\/', '', regex=True)

#1-thread TEST
%%time
for street in df_streets[:200]['EGRP365_STREETS_CLEAN']:
    street_contains_idx = df[(df['STR'].str.contains(street, case=False))].index
    df_txt = df.loc[street_contains_idx]
    #print(len(df_txt[df_txt['L0B1']==1]), len(street_contains_idx), '\t', street)

#Multi-thread TEST
def street_contains(df_streets):
    for street in df_streets:
        street_contains_idx = df[(df['STR'].str.contains(street, case=False))].index
        df_txt = df.loc[street_contains_idx]
        #print(len(df_txt[df_txt['L0B1']==1]), len(street_contains_idx), '\t', street, '_____', pool.Process())
    return df_txt

In [18]:
def street_contains2(df_streets):
    start_time = datetime.datetime.now()
    print('LAND & OKS connects detect start:', start_time.strftime('%Y-%m-%d %H:%M:%S'), '_____', pool.Process())
    df_merge_all = pd.DataFrame(columns=['L0B1_1','L0B1_2','addr1','addr2','cad1','cad2','area1','area2','ord1','ord2','inegrp1','inegrp2'])

    for street in df_streets:

        street_contains_idx = df[df['STR'].str.contains(street, case=False)].index
        df_txt = df.loc[street_contains_idx]
        ##print(len(df_txt[df_txt['L0B1']==1]), len(street_contains_idx), '\t', street)
        #print(len(street_contains_idx), '\t', street)

        if (len(df_txt[df_txt['L0B1']==1])>0) & (len(df_txt)>1): # Если в датафрейме, группированном по улицам есть хотя-бы 1 здание
            df_txt = df_txt.reset_index()
            #print('table', df_txt.shape, 'street:', street)

            cv = CountVectorizer(ngram_range=(1, 1))
            cv_fit=cv.fit_transform(df_txt['STR'])
            cv_fit_texts = cv_fit.toarray()
            #print('vector', cv_fit_texts.shape)

            mergings = linkage(cv_fit_texts, method='ward', metric='euclidean')
            #print('distance', mergings.shape)

            df_merg = pd.DataFrame(mergings, columns=['clust1','clust2','range','clust_count'])
            df_merg[['clust1','clust2','clust_count']] = df_merg[['clust1','clust2','clust_count']].astype(int)
            df_merg0 = df_merg[ (df_merg['range']<=5) & (df_merg['clust_count']<=2) & (df_merg['clust1']<=cv_fit_texts.shape[0]) & (df_merg['clust2']<=cv_fit_texts.shape[0]) ]
            #print('connect parcel&oks?', df_merg0.shape)

            df_merg0['L0B1_1'] = df_txt['L0B1'].copy().loc[df_merg0['clust1']].values
            df_merg0['L0B1_2'] = df_txt['L0B1'].copy().loc[df_merg0['clust2']].values
            df_merg0['addr1'] = df_txt['ADDRESS'].copy().loc[df_merg0['clust1']].values
            df_merg0['addr2'] = df_txt['ADDRESS'].copy().loc[df_merg0['clust2']].values
            df_merg0['cad1'] = df_txt['CADNUM'].copy().loc[df_merg0['clust1']].values
            df_merg0['cad2'] = df_txt['CADNUM'].copy().loc[df_merg0['clust2']].values
            df_merg0['area1'] = df_txt['AREA'].copy().loc[df_merg0['clust1']].values
            df_merg0['area2'] = df_txt['AREA'].copy().loc[df_merg0['clust2']].values
            df_merg0['ord1'] = df_txt['ORDINATE'].copy().loc[df_merg0['clust1']].values
            df_merg0['ord2'] = df_txt['ORDINATE'].copy().loc[df_merg0['clust2']].values
            df_merg0['inegrp1'] = df_txt['INEGRP'].copy().loc[df_merg0['clust1']].values
            df_merg0['inegrp2'] = df_txt['INEGRP'].copy().loc[df_merg0['clust2']].values

            #print( 'with only one in egrp:', len( df_merg0[ ~((df_merg0['inegrp1']=='1')&(df_merg0['inegrp2']=='1')) ] ), '\n')
            df_merge_all = pd.concat([df_merge_all, df_merg0])

    df_merge_all.to_csv('df_merge_all.csv', sep=';', encoding = 'cp1251', index=False)

    stop_time = datetime.datetime.now()
    print('Saved doubles: ', len(df_merge_all), 'LAND & OKS serch links stop :', stop_time.strftime('%Y-%m-%d %H:%M:%S'), ' Total sec:', int(datetime.timedelta.total_seconds(stop_time - start_time)), '_____', pool.Process())
    return df_merge_all

In [ ]:
%%time
# Multiprocessing linkage clusterisation in street groups
try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default
print('Use', cpus, 'threads')

pool = multiprocessing.Pool(processes=cpus)

df33 = pd.concat(pool.map( street_contains2, np.array_split(df_streets['EGRP365_STREETS_CLEAN'], cpus) ))

pool.terminate()

Use 4 threads


In [254]:
#TEST Process start time / stop time console out
start_time = datetime.datetime.now()
print('LAND & OKS connects detect start:', start_time.strftime('%Y-%m-%d %H:%M:%S'))
time.sleep(3)
stop_time = datetime.datetime.now()
print('LAND & OKS connects detect stop :', stop_time.strftime('%Y-%m-%d %H:%M:%S'), ' Total sec:', int(datetime.timedelta.total_seconds(stop_time - start_time)))

LAND & OKS connects detect start: 2018-08-29 14:27:57
LAND & OKS connects detect stop : 2018-08-29 14:28:00  Total sec: 3


In [191]:
# One thread linkage in street group
start_time = datetime.datetime.now()
print('LAND & OKS connects detect start:', start_time.strftime('%Y-%m-%d %H:%M:%S'))

df_merge_all = pd.DataFrame(columns=['L0B1_1','L0B1_2','addr1','addr2','cad1','cad2','area1','area2','ord1','ord2','inegrp1','inegrp2'])

for street in df_streets['EGRP365_STREETS_CLEAN']:

    street_contains_idx = df[df['STR'].str.contains(street, case=False)].index
    df_txt = df.loc[street_contains_idx]
    print(len(street_contains_idx), '\t', street)

    if (len(df_txt[df_txt['L0B1']==1])>0) & (len(df_txt)>1): # Если в датафрейме, группированном по улицам есть хотя-бы 1 здание
        df_txt = df_txt.reset_index()
        print('table', df_txt.shape, 'street:', street)

        cv = CountVectorizer(ngram_range=(1, 1))
        cv_fit=cv.fit_transform(df_txt['STR'])
        cv_fit_texts = cv_fit.toarray()
        print('vector', cv_fit_texts.shape)

        mergings = linkage(cv_fit_texts, method='ward', metric='euclidean')
        print('distance', mergings.shape)

        df_merg = pd.DataFrame(mergings, columns=['clust1','clust2','range','clust_count'])
        df_merg[['clust1','clust2','clust_count']] = df_merg[['clust1','clust2','clust_count']].astype(int)
        df_merg0 = df_merg[ (df_merg['range']<=5) & (df_merg['clust_count']<=2) & (df_merg['clust1']<=cv_fit_texts.shape[0]) & (df_merg['clust2']<=cv_fit_texts.shape[0]) ]
        print('connect parcel&oks?', df_merg0.shape)

        df_merg0['L0B1_1'] = df_txt['L0B1'].copy().loc[df_merg0['clust1']].values
        df_merg0['L0B1_2'] = df_txt['L0B1'].copy().loc[df_merg0['clust2']].values
        df_merg0['addr1'] = df_txt['ADDRESS'].copy().loc[df_merg0['clust1']].values
        df_merg0['addr2'] = df_txt['ADDRESS'].copy().loc[df_merg0['clust2']].values
        df_merg0['cad1'] = df_txt['CADNUM'].copy().loc[df_merg0['clust1']].values
        df_merg0['cad2'] = df_txt['CADNUM'].copy().loc[df_merg0['clust2']].values
        df_merg0['area1'] = df_txt['AREA'].copy().loc[df_merg0['clust1']].values
        df_merg0['area2'] = df_txt['AREA'].copy().loc[df_merg0['clust2']].values
        df_merg0['ord1'] = df_txt['ORDINATE'].copy().loc[df_merg0['clust1']].values
        df_merg0['ord2'] = df_txt['ORDINATE'].copy().loc[df_merg0['clust2']].values
        df_merg0['inegrp1'] = df_txt['INEGRP'].copy().loc[df_merg0['clust1']].values
        df_merg0['inegrp2'] = df_txt['INEGRP'].copy().loc[df_merg0['clust2']].values

        #print( 'with only one in egrp:', len( df_merg0[ ~((df_merg0['inegrp1']=='1')&(df_merg0['inegrp2']=='1')) ] ), '\n')
        df_merge_all = pd.concat([df_merge_all, df_merg0])

df_merge_all.to_csv('df_merge_all.csv', sep=';', encoding = 'cp1251', index=False)
print('Saved doubles: ', len(df_merge_all), '\n')

stop_time = datetime.datetime.now()
print('LAND & OKS connects detect stop :', stop_time.strftime('%Y-%m-%d %H:%M:%S'), ' Total sec:', int(datetime.timedelta.total_seconds(stop_time - start_time)))

LAND & OKS connects detect start: 2018-08-28 18:18:12
23 	 аллея 1 маевки
table (23, 14) street: аллея 1 маевки
vector (23, 55)
distance (22, 4)
connect parcel&oks? (6, 4)


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

0 	 проезд 1-й ходынский
2 	 улица 10-летия октября
0 	 улица 1812 года  1812 года
0 	 улица 1905 года  1905 года
0 	 проезд 2-й ходынский
3 	 улица 26-ти бакинских комиссаров
0 	 проезд 3-й ходынский
0 	 проезд 4-й ходынский
8 	 проспект 40 лет октября
6 	 улица 50 лет октября
2 	 проспект 60-летия октября
1 	 площадь 60-летия ссср
5 	 улица 8 марта
5 	 улица 8 марта
0 	 улица 8 марта 1-я
0 	 улица 8 марта 4-я
3 	 улица 800-летия москвы
4 	 улица 9 мая
0 	 улица абельмановская
5 	 площадь абельмановская застава
0 	 улица абрамцевская
1 	 просека абрамцевская
0 	 переулок абрикосовский
0 	 улица авангардная
0 	 улица авиаконструктора микояна
2 	 улица авиаконструктора миля
0 	 улица авиаконструктора петлякова  улица авиаконструктора петлякова
1 	 улица авиаконструктора сухого
1 	 улица авиамоторная
0 	 проезд авиамоторный
8 	 улица авиаторов
1 	 улица авиационная
0 	 переулок авиационный
0 	 площадь автозаводская
1 	 улица автозаводская
0 	 проезд автозаводский 1-й  1-й автозаводский
0

1 	 улица береговая
53 	 проезд береговой
table (53, 14) street: проезд береговой
vector (53, 68)
distance (52, 4)
connect parcel&oks? (13, 4)
0 	 набережная бережковская
0 	 аллея березовая  берёзовая
0 	 проезд березовой рощи  берёзовой рощи
22 	 улица берзарина
39 	 проезд берингов
table (39, 14) street: проезд берингов
vector (39, 41)
distance (38, 4)
connect parcel&oks? (5, 4)
0 	 проезд берингов н.  новый берингов
0 	 переулок берников
0 	 набережная берниковская
0 	 набережная берсеневская
0 	 переулок берсеневский
0 	 шоссе бесединское
0 	 бульвар бескудниковский
0 	 переулок бескудниковский
14 	 проезд бескудниковский
table (14, 14) street: проезд бескудниковский
vector (14, 32)
distance (13, 4)
connect parcel&oks? (5, 4)
12 	 улица бестужевых
10 	 улица бехтерева
1 	 улица бибиревская
0 	 улица библиотечная
11 	 проезд библиотечный
table (11, 14) street: проезд библиотечный
vector (11, 47)
distance (10, 4)
connect parcel&oks? (5, 4)
0 	 площадь биржевая
0 	 улица бирюлевская 

0 	 территория внуково аэропорт
0 	 улица внуковская 1-я  1-я внуковская
0 	 улица внуковская 2-я  2-я внуковская
0 	 улица внуковская 3-я  3-я внуковская
0 	 улица внуковская 4-я  4-я внуковская
0 	 улица внуковская 5-я  5-я внуковская
0 	 улица внуковская б.  большая внуковская
0 	 шоссе внуковское
0 	 километр внуковское шоссе 5-й п внуковское  внуковское шоссе, 5-й п внуковское
13 	 проезд внутренний
table (13, 14) street: проезд внутренний
vector (13, 67)
distance (12, 4)
connect parcel&oks? (4, 4)
0 	 улица водников
0 	 переулок водопроводный
0 	 улица водопьянова
7 	 улица воздвиженка
0 	 улица воздушная
0 	 переулок вознесенский
0 	 проезд вознесенский
1 	 улица войкова
0 	 проезд войковский 1-й  1-й войковский
0 	 проезд войковский 2-й  2-й войковский
0 	 проезд войковский 3-й  3-й войковский
0 	 проезд войковский 4-й  4-й войковский
0 	 проезд войковский 5-й  5-й войковский
0 	 улица вокзальная
0 	 площадь вокзальная
0 	 переулок вокзальный
2 	 проспект волгоградский
1 	 буль

...

10 	 платформа северянин
20 	 проезд северянинский
table (20, 14) street: проезд северянинский
vector (20, 55)
distance (19, 4)
connect parcel&oks? (6, 4)
0 	 улица седова
0 	 улица селезневская  селезнёвская
0 	 переулок селиверстов  селивёрстов
0 	 улица селигерская
6 	 улица сельскохозяйственная
0 	 проезд сельскохозяйственный 1-й  1-й сельскохозяйственный
6 	 проезд сельскохозяйственный 2-й
table (6, 14) street: проезд сельскохозяйственный 2-й
vector (6, 30)
distance (5, 4)
connect parcel&oks? (2, 4)
0 	 площадь семеновская  семёновская
0 	 набережная семеновская  семёновская
0 	 улица семеновская б.  большая семёновская
0 	 улица семеновская м.  малая семёновская
0 	 переулок семеновский  семёновский
1 	 проезд семеновский
0 	 улица семеновский вал  семёновский вал
0 	 тупик семинарский
0 	 улица сенежская
3 	 улица серафимовича
0 	 улица сергея капицы  улица сергея капицы
6 	 улица сергея макеева
3 	 улица сергея эйзенштейна
0 	 переулок сергиевский б.  большой сергиевский
0 	 пе

0 	 улица старослободская
0 	 переулок старослободский
0 	 улица староспасская
0 	 улица старофилинская
0 	 улица стартовая
1 	 улица старые кузьминки
3 	 улица старый гай
0 	 улица стасовой
0 	 улица стахановская
0 	 улица стекольная 1-я  1-я стекольная
2 	 улица степана супруна
0 	 улица степана шутова
3 	 улица столетова
1 	 переулок столешников
0 	 переулок столовый
0 	 улица стольника потемкина  улица стольника потёмкина
0 	 переулок столярный
0 	 улица сторожевая
0 	 бульвар страстной
35 	 проезд стратонавтов
table (35, 14) street: проезд стратонавтов
vector (35, 62)
distance (34, 4)
connect parcel&oks? (12, 4)
0 	 улица страховская
0 	 улица стрелецкая
0 	 проезд стрелецкий 1-й  1-й стрелецкий
0 	 проезд стрелецкий 2-й  2-й стрелецкий
0 	 проезд стрелецкий 3-й  3-й стрелецкий
0 	 проезд стрелецкий 4-й  4-й стрелецкий
1 	 переулок стрельбищенский
0 	 переулок стремянный
2 	 проезд строгановский
0 	 бульвар строгинский
0 	 шоссе строгинское
3 	 улица строителей
143 	 проезд строит

0 	 улица тюрина
1 	 аллея тютчевская
0 	 улица тюфелева роща
0 	 переулок уваровский
0 	 улица угличская
0 	 переулок угловой
0 	 улица угрешская
0 	 проезд угрешский 1-й  1-й угрешский
0 	 проезд угрешский 2-й  2-й угрешский
0 	 проезд угрешский 3-й  3-й угрешский
20 	 улица удальцова
0 	 переулок узкий
0 	 бульвар украинский
1 	 переулок уланский
1 	 улица улофа пальме
0 	 площадь университетская
0 	 проспект университетский
0 	 переулок упорный
1 	 улица уральская
0 	 улица уржумская
0 	 улица усачева  усачёва
0 	 переулок усачевский  усачёвский
3 	 улица усиевича
0 	 улица усиевича 2-я
0 	 переулок успенский
0 	 улица уссурийская
0 	 набережная устьинская
8 	 проезд устьинский
0 	 улица уткина
0 	 улица утренняя
0 	 улица ухтомская
0 	 переулок ухтомский
0 	 улица ухтомского ополчения
0 	 переулок учебный
0 	 улица учинская
3 	 улица фабрициуса
1 	 улица фабричная
4 	 проезд фабричный
4 	 улица фадеева
table (4, 14) street: улица фадеева
vector (4, 30)
distance (3, 4)
connect parc

0 	 аллея чоботовская 2-я  2-я чоботовская
0 	 аллея чоботовская 3-я  3-я чоботовская
0 	 аллея чоботовская 4-я  4-я чоботовская
0 	 аллея чоботовская 5-я  5-я чоботовская
0 	 аллея чоботовская 6-я  6-я чоботовская
0 	 аллея чоботовская 7-я  7-я чоботовская
0 	 аллея чоботовская 8-я  8-я чоботовская
0 	 аллея чоботовская 9-я  9-я чоботовская
8 	 проезд чоботовский
table (8, 14) street: проезд чоботовский
vector (8, 29)
distance (7, 4)
connect parcel&oks? (2, 4)
0 	 бульвар чонгарский
2 	 улица чугунные ворота
0 	 переулок чудов б.  большой чудов
28 	 проезд чукотский
table (28, 14) street: проезд чукотский
vector (28, 41)
distance (27, 4)
connect parcel&oks? (5, 4)
0 	 тупик чуксин
0 	 улица чусовская
10 	 улица шаболовка
0 	 улица шарикоподшипниковская
1 	 площадь шарля де голля
0 	 улица шатурская
0 	 тупик шведский
4 	 улица шверника
table (4, 14) street: улица шверника
vector (4, 13)
distance (3, 4)
connect parcel&oks? (2, 4)
0 	 проезд шебашевский  шебашёвский
0 	 переулок шебашев

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [192]:
df_merge_all[((df_merge_all['L0B1_1']==0) & (df_merge_all['L0B1_2']==1)) | ((df_merge_all['L0B1_1']==1) & (df_merge_all['L0B1_2']==0))]

,L0B1_1,L0B1_2,addr1,addr2,area1,area2,cad1,cad2,clust1,clust2,clust_count,inegrp1,inegrp2,ord1,ord2,range
4,1,0,"ул Адмирала Макарова, д 2Б, строение Б/Н, (г.МОСКВА, УЛИЦА АДМИРАЛА МАКАРОВА, вл. 2Б, стр.Б/Н)","ул Адмирала Макарова, вл 45, строен 1, (г Москва, улица Адмирала Макарова, вл. 45, стр.1)",15.6,1170.00,77:09:0001023:4407,77:09:0001021:29,0.0,5.0,2.0,1,0,0,1,3.316625
59,1,0,"р-н Лосиноостровский, проезд Анадырский, д 51, корп 2","проезд Анадырский, д 47, корп 1",5783.0,30.00,77:02:0010005:1002,77:02:0010008:35,0.0,113.0,2.0,0,0,1,1,1.732051
61,1,0,"проезд Анадырский, д 65, (г.Москва, пр.Анадырский, д.65)","проезд Анадырский, д 65, (г Москва, проезд Анадырский, вл 65)",2869.4,10671.00,77:02:0010005:6552,77:02:0010005:54,19.0,145.0,2.0,1,1,0,1,1.732051
82,1,0,"проезд Анадырский, д 79, строение 1, (г.Москва, пр.Анадырский, д.79, строен.1)","проезд Анадырский, вл 79, строен 3, (г. Москва, пр. Анадырский, вл. 79, стр. 3)",5449.6,36.00,77:02:0012002:4727,77:02:0012002:3918,6.0,97.0,2.0,1,0,0,1,2.449490
6,1,0,"проезд Аэропорта, д 8, строен 10, (г. Москва, пр. Аэропорта, д. 8, стр. 10)","проезд Аэропорта, д 10, (г Москва, проезд Аэропорта, вл 10)",341.6,8575.00,77:09:0005004:9065,77:09:0005004:54,2.0,18.0,2.0,0,1,0,1,2.236068
27,1,0,"р-н Марьино, проезд Батайский, д 5, строен 2","проезд Батайский, д 65, строен 2, (г Москва, проезд Батайский, д 65, стр 2)",257.5,306.00,77:04:0004019:1181,77:04:0004019:33,0.0,23.0,2.0,1,0,1,1,3.000000
14,1,0,"р-н Беговой, аллея Беговая, д 3а, строен 2","аллея Беговая, (г Москва, Ленинградский проспект, от ул Беговая до м Сокол)",336.8,35412.39,77:09:0005015:1163,77:09:0005008:94,1.0,16.0,2.0,1,0,1,1,3.316625
34,1,0,"проезд Береговой, д 2А, (г.Москва, пр.Береговой, д.2А)","проезд Береговой, вл 2А, (г. Москва, Береговой проезд, вл. 2А)",137.7,380.00,77:07:0002003:7550,77:07:0002003:1,4.0,17.0,2.0,1,1,0,1,2.449490
21,1,0,"р-н Свиблово, проезд Берингов, д 1, корп 1","проезд Берингов, д 6, корп 1",158.1,30.00,77:02:0015006:1017,77:02:0015006:59,2.0,13.0,2.0,1,0,1,1,1.000000
29,1,0,"р-н Мещанский, ул Лубянка Б., д 7, строен 3, (г Москва, Улица Большая Лубянка, д 7, строен 3)","ул Лубянка Б., д 2, (г.МОСКВА, УЛИЦА БОЛЬШАЯ ЛУБЯНКА дом 2)",274.1,8690.00,77:01:0001039:1638,77:01:0001036:16,11.0,72.0,2.0,1,1,1,1,2.449490


In [207]:
#df_merge_all['L+B'] = df_merge_all['L0B1_1'] + df_merge_all['L0B1_2']
df_merge_all[ (df_merge_all['L+B']==1) & (df_merge_all['cad1'].str[:-5]==df_merge_all['cad2'].str[:-5]) ].sort_valuees(['range'])

,L0B1_1,L0B1_2,addr1,addr2,area1,area2,cad1,cad2,clust1,clust2,clust_count,inegrp1,inegrp2,ord1,ord2,range,L+B
1,1,0,"проезд Днепропетровский, д 1, (г Москва, проезд Днепропетровский, д 1)","проезд Днепропетровский, (г. Москва, проезд Днепропетровский)",11962.6,60956.0,77:05:0007003:2744,77:05:0007003:1002,6.0,13.0,2.0,1,1,0,1,0.000000,1
0,1,0,"аллея Рассветная, д 2, (г. Москва, Рассветная аллея, д. 2)","аллея Рассветная, (г. Москва, Рассветная аллея)",1049.6,17886.0,77:03:0007001:1037,77:03:0007001:1418,0.0,9.0,2.0,0,1,0,1,0.000000,1
3,1,0,"проезд Перова Поля 2-й, д 7, (г. Москва, 2-й пр. Перова поля, д. 7)","проезд Перова Поля 2-й, д 7, (г. Москва, 2-й пр. Перова Поля, вл.7)",972.7,4579.0,77:03:0006008:1155,77:03:0006008:3307,0.0,8.0,2.0,1,1,0,1,1.000000,1
7,1,0,"проезд Физкультурный, д 4, строение 9, (г.Москва, пр.Физкультурный, д.4, строен.9)","проезд Физкультурный, д 3, корп 2, строен 1, (г. Москва, Физкультурный пр-д, д. 3, к. 2, стр. 1)",42.6,70.0,77:07:0005008:5693,77:07:0005008:5683,0.0,11.0,2.0,1,0,0,1,1.732051,1
1,1,0,"проезд Извилистый, д 9, строение 1, (г.Москва, пр.Извилистый, д.9, стр.1)","проезд Извилистый, (г. Москва, Извилистый пр., вл. 7)",2577.5,7253.0,77:02:0011003:5213,77:02:0011003:1007,1.0,8.0,2.0,1,0,0,1,1.732051,1
0,1,0,"проезд СОКОЛЬНИЧЕСКИЙ ПАВИЛЬОННЫЙ, д ВЛ.2, строение 1, (г.Москва, пр.СОКОЛЬНИЧЕСКИЙ ПАВИЛЬОННЫЙ, д.ВЛ.2, строен.1)","проезд Сокольнический Павильонный, д 2, строен 1, (г. Москва, пр. Сокольнический Павильонный, вл. 2, стр. 1)",92.5,109.0,77:03:0001001:3029,77:03:0001001:2784,0.0,1.0,2.0,1,0,0,1,1.732051,1
31,1,0,"проезд Старопетровский, д 11, корп 1, (г.Москва, пр.Старопетровский, д.11, корп.1)","проезд Старопетровский, д 11, (г. Москва, Старопетровский пр., 11)",15234.2,50658.0,77:09:0003013:2367,77:09:0003013:2816,13.0,22.0,2.0,1,0,0,1,2.000000,1
3,1,0,"проезд Красноворотский, д 3Б, строение 3, (г. Москва, пр. Красноворотский, д. 3Б, стр. 3)","проезд Красноворотский, вл 3Б, строение 2, (г. Москва, Красноворотский пр., вл. 3Б, стр. 2)",950.6,491.0,77:01:0003040:5353,77:01:0003040:5303,2.0,4.0,2.0,1,1,0,1,2.000000,1
40,1,0,"проезд Нагорный, д 10, корп 1, (г.Москва, пр.Нагорный, д.10, корп.1)","проезд Нагорный, д 10, (г. Москва, Нагорный пр-д, д.10)",1035.3,42853.0,77:05:0003001:1872,77:05:0003001:1805,25.0,63.0,2.0,1,0,0,1,2.000000,1
10,1,0,"проезд Причальный, д 6, корп 1, (г.Москва, пр.Причальный, д.6, корп.1)","проезд Причальный, вл 4, корп 2, (г. Москва, Причальный пр., вл. 4, корп. 2)",1046.2,1481.0,77:08:0012005:1980,77:08:0012005:1681,1.0,11.0,2.0,0,0,0,1,2.000000,1


In [195]:
df_merge_all.to_excel(u'Land&Build_links_2.xlsx', index=False)

In [72]:
### df = df.replace({'STREETNM': {' М\.':' Мал.', ' Б\.':' Больш.'}}, regex=True)

In [ ]:
### df[df['STREETNM'].str.contains('Горько', case=True)]

In [75]:
len(df[df['ADDRESS'].isnull()])

0

In [76]:
df['ADDRESS'].fillna('', inplace=True)

In [77]:
### df[df['STREETNM'] = df['STREETNM'].str.replace('В', 'мкр В')

In [ ]:
df[(df['STREETNM']=='В')]

In [82]:
#TEST
df['ADDRESS'] = df['ADDRESS'].replace(r'\(+','[', regex=True).replace(r'\)+',']', regex=True).replace(r'\*+','`', regex=True).replace(r'\~+','-', regex=True).replace(r'\'+','"', regex=True).replace(r'\_+','', regex=True).replace(r'\;+',',', regex=True).replace(r'\\' ,'/', regex=True)

CPU times: user 11.6 s, sys: 204 ms, total: 11.8 s
Wall time: 11.7 s


#### Вывод минимальных расстояний в кластере

In [68]:
df_merg0 = df_merg[ (df_merg['range']<=0.5) & (df_merg['clust1']<=22899) & (df_merg['clust2']<=22899) ]
df_merg0.shape

(62, 4)

In [ ]:
%%time
#Строим дендрограмму, указав параметры удобные для отображения
fig = plt.subplots(figsize=(25,240))

#dendrogram(mergings, orientation='left', labels=np.array(np.arange(0,len(cv_fit_texts))), no_labels=False, show_leaf_counts=True, leaf_rotation=0, leaf_font_size=7, distance_sort=True)
dendrogram(mergings, p=100, truncate_mode='level', count_sort='ascending', orientation='left', labels=np.array(df_txt[['AREA','OKATO','STR']]), no_labels=False, show_leaf_counts=True, leaf_rotation=0, leaf_font_size=8, distance_sort=True)

plt.show()

#### Кластеризация KMeans

In [203]:
# Для кластеризации импортируем KMeans
from sklearn.cluster import KMeans

In [205]:
%%time
df_txt_kmeans = KMeans(n_clusters=2, random_state=0).fit(cv_fit_texts)
df_txt['class'] = df_txt_kmeans.labels_

Wall time: 1.84 s
